In [13]:
import os
import random
import silero
from scipy.signal import convolve
import numpy as np
import pandas as pd
import librosa
from glob import glob
from IPython.display import display, Audio
import math

SAMPLE_RATE = 16000
RIR_files = os.listdir('RIR/')[1:]
BG_files = os.listdir('Bg_sound')

In [14]:
def get_audio_list(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory)]

def get_org_audfname(aud_dir, fname):
    return os.path.join(aud_dir, fname.split('/')[-1][2:])

def get_context_label(fname):
    return (int)(fname.split('/')[-1][0])

def normalise_npArray(arr):
    arr = (arr - min(arr))/(max(arr) - min(arr))
    return arr

def get_convolved_audio(aud_dir, filename):
    
    label = get_context_label(filename)
    org_audio_fname = get_org_audfname(aud_dir, filename)
    rir_fname = os.path.join('RIR/', RIR_files[label])
    
    org_wav, sr = librosa.load(org_audio_fname, sr=SAMPLE_RATE)
    #org_wav = normalise_npArray(org_wav)
    rir_wav, sr = librosa.load(rir_fname, sr=SAMPLE_RATE)
    #rir_wav = normalise_npArray(rir_wav)

    convolved_audio = convolve(org_wav, rir_wav)[0:org_wav.shape[0]]
    #convolved_audio = normalise_npArray(convolved_audio)
    return convolved_audio

def get_noise_from_sound(signal,noise,SNR):
    
    RMS_s=math.sqrt(np.mean(signal**2))
    #required RMS of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    
    #current RMS of noise
    RMS_n_current=math.sqrt(np.mean(noise**2))
    if RMS_n_current == 0:
        RMS_n_current = 1
    noise=noise*(RMS_n/RMS_n_current)
    
    return noise

def add_noise(audio, noise):

    SNR_list = [0, 5, 10, 15, 20]
    SNR_choice = random.choice(SNR_list)
    #print(SNR_choice)
    noise = get_noise_from_sound(audio, noise[0:audio.shape[0]], SNR_choice)
    noise = normalise_npArray(noise)
    noisy_audio = audio + noise
    return noisy_audio

def get_noisy_audio(aud_dir, aud_fname):
    label = get_context_label(aud_fname)

    convolved_audio = get_convolved_audio(aud_dir, aud_fname)
    bg_file = os.path.join('Bg_sound/', BG_files[2-label])
    bg_wav, sr = librosa.load(bg_file, sr=SAMPLE_RATE)
    noisy_audio = add_noise(convolved_audio, bg_wav)
    #noisy_audio = normalise_npArray(noisy_audio)

    return noisy_audio

In [5]:
aud_dir = '../../LibriVox_Kaggle/achtgesichterambiwasse/'
aud_list = get_audio_list(r'../../LibriVox_Kaggle/achtgesichterambiwasse/n1n2/')
aud_list.sort()
aud_list.pop()
get_context_label(aud_list[1]), aud_list[1], get_org_audfname(aud_dir, aud_list[1])

(0,
 '../../LibriVox_Kaggle/achtgesichterambiwasse/n1n2/0_achtgesichterambiwasse_0002.wav',
 '../../LibriVox_Kaggle/achtgesichterambiwasse/achtgesichterambiwasse_0002.wav')

In [11]:
import torch
import torchaudio

model, decoder, utils = silero.silero_stt(language='de', version='latest', jit_model='jit')

(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils
device = torch.device('cuda:0')
model = model.to(device)

def sileromodel(file):

    test_files = glob(file)
    batches = split_into_batches(test_files, batch_size=10)
    input = prepare_model_input(read_batch(batches[0]),
                            device=device)

    output = model(input)
    output = output.cpu()

    for example in output:
        out = decoder(example.cpu())

    #print(out)

    return out

In [10]:
aud_file = random.choice(aud_list)

sileromodel(aud_file)

und zeichnete nachher die roten korallenäste des feuerflackerns


/work/dpandya/miniconda3/envs/irsolve/lib/python3.10/site-packages/torch/nn/modules/module.py:1194: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return forward_call(*input, **kwargs)


'und zeichnete nachher die roten korallenäste des feuerflackerns'

In [15]:
silero_preds = []

for i in range(0, len(aud_list)):
    noisy_audio = get_noisy_audio(aud_dir, aud_list[i])
    preds = silero
    silero_preds.append(sileromodel(noisy_audio))
    print(i)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()